Ta skripta se uporablja za pridobitev rezultatov dinamičnih simulacij.

In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
sys.path.append(os.path.join(os.getcwd(), "..")) # Add the parent directory to the path

# Import the PowerFactory API
from pf_python_api import PowerFactoryApp

# Select the project to open (can also be opened from the GUI when it starts)
project_name = "SLO_EES_2024_projekt_SSSC"

# Start PowerFactory, load the project and show the GUI (non-interactive mode)
app = PowerFactoryApp()
app.activate_project(project_name)
app.show_gui()

PowerFactory application started successfully!
Project SLO_EES_2024_projekt_SSSC successfully activated


In [2]:
# Get the ElmRes object
elmRes = app.get_calc_relevant_objects("*All calculations.ElmRes")[0]
print("Ime Rezultatov: ", elmRes.GetAttribute("loc_name"))

Ime Rezultatov:  All calculations


In [ ]:
from typing import List
from powerfactory import DataObject

# Obtain all generators
all_generators = app.app.GetCalcRelevantObjects("*.ElmSym", 1, 1, 1)
print("All generators: ", len(all_generators))

# Obtain only energized generators
generators: List[DataObject] = []
for gen in all_generators:
    if gen.GetAttribute("outserv") == 1:
        continue
    if gen.IsEnergized() != 1:
        continue
    generators.append(gen)
print("Active Generators: ", len(generators))

# UPDATE Generators order to match Tadej's order
import pandas as pd
data = pd.read_excel('Vrstni_red_GEN.xlsx')
pf_names = data['PF_NAME'].values
area = data['AREA'].values

# Filter and reorder generators to match the order from pf_names
ordered_generators = [gen for name in pf_names for gen in generators if gen.GetAttribute("loc_name") == name]
print("Ordered Generators: ", len(ordered_generators))

# Get list of names where area is "SLO"
slo_names = [name for name, a in zip(pf_names, area) if a == "SLO"]

# Filter ordered_generators to include only those with "SLO" under area
slo_generators = [gen for name in slo_names for gen in generators if gen.GetAttribute("loc_name") == name]
print("SLO Generators: ", len(slo_generators))

All generators:  126
Active Generators:  56
Ordered Generators:  56
SLO Generators:  18


In [ ]:
'''
Prepare the ElmRes object for the results
'''
# Obtain the ElmRes object
elmRes = app.get_calc_relevant_objects("*Izvoz RoCof Martin.ElmRes")[0]
print("Ime Rezultatov: ", elmRes.GetAttribute("loc_name"))

# Add required variables to the ElmRes object for each generator
for gen in generators:
    obj = elmRes.CreateObject("IntMon")
    obj.SetAttribute("loc_name", gen.GetAttribute("loc_name"))
    obj.SetAttribute("obj_id", gen)
    selected_variables = ["s:P1"]
    obj.SetAttribute("vars", selected_variables)

Ime Rezultatov:  Izvoz RoCof Martin


In [5]:
# DISABLE ALL CURRENT SIMULATION EVENTS (set as out of service)
active = app.app.GetActiveStudyCase()
eventFolder: DataObject = next((con for con in active.GetContents() if con.loc_name == "Simulation Events/Fault"))
simulation_events: List[DataObject] = eventFolder.GetContents()
for event in simulation_events:
    event.SetAttribute("outserv", 1)

In [6]:
import time

iteration = 0
created_events = []
previous_event = None
files_names = []

for gen in ordered_generators:
    iteration += 1

    gen_name = gen.GetAttribute("loc_name")
    print(f"Processing generator outage for generator nr. {iteration}: {gen_name}")

    # ====== 1. We define the SwitchEvent
    new_event = eventFolder.CreateObject("EvtSwitch")
    new_event.SetAttribute("loc_name", gen_name+"_izpad")
    new_event.SetAttribute("p_target", gen)
    new_event.SetAttribute("time", 2)
    created_events.append(new_event)

    # ====== 2. We run the simulation
    # Calculate initial conditions
    oInit = app.app.GetFromStudyCase('ComInc')  # Get initial condition calculation object
    oInit.Execute()

    # Run RMS-simulation
    oRms = app.app.GetFromStudyCase('ComSim')   # Get RMS-simulation object
    oRms.Execute()

    # ====== 3. We delete the current event if it exists
    if previous_event is not None:
        deleted = previous_event.Delete()
        if deleted == 0:
            pass
        else:
            print("Failed to delete event: ", previous_event.GetAttribute("loc_name"))
    new_event.SetAttribute("outserv", 1)

    # ====== 4. We get the results
    comRes = app.app.GetFromStudyCase("ComRes")
    comRes.pResult = elmRes         # Set ElmRes object to export from # type: ignore
    comRes.iopt_exp = 6             # Set export to csv - 6 # type: ignore
    comRes.iopt_sep = 0             # Set use the system seperator # type: ignore
    comRes.iopt_honly = 0           # To export data and not only the head er # type:ignore
    comRes.iopt_csel = 1            # Set export to only selected variables # type: ignore
    comRes.numberPrecisionFixed = 8 # Set the number precision to 6 decimals # type: ignore
    comRes.col_Sep = ";"            # Set the column separator to ; # type: ignore
    comRes.dec_Sep = ","            # Set the decimal separator to , # type: ignore

    # Set File path and name 
    file_name = "results_izpad_" + gen_name + ".csv"
    files_names.append(file_name)
    results_folder = r"C:\Projects\PowerFactory\slovenia_ees\rms_results\martin_scenario"
    if not os.path.exists(results_folder):
        os.makedirs(results_folder)
    file_path = os.path.join(results_folder, file_name)
    comRes.f_name = file_path # type: ignore

    resultObject = [None] # type: ignore
    elements = [elmRes] # type: ignore
    variable = ["b:tnow"] # type: ignore

    for gen in generators:
        resultObject.append(None)
        elements.append(gen)
        variable.append("s:P1")
    
    # Set the selected variables
    comRes.resultobj = resultObject # Export selected # type: ignore
    comRes.element = elements # type: ignore
    comRes.variable = variable # type: ignore

    # Export the results
    comRes.Execute() # type: ignore 

    # Await for the file to be accessible
    while not os.path.exists(file_path):
        time.sleep(0.1)

    # ====== 5. We disable the SwitchEvent (set to out of service here as cannot delete, will be deleted in the next iteration)
    new_event.SetAttribute("outserv", 1)
    previous_event = new_event

    app.app.ClearOutputWindow()

Processing generator outage for generator nr. 1: Doblar G2
Processing generator outage for generator nr. 2: Plave G1
Processing generator outage for generator nr. 3: Plave G2
Processing generator outage for generator nr. 4: Solkan G1
Processing generator outage for generator nr. 5: HE Moste G1
Processing generator outage for generator nr. 6: Medvode G1
Processing generator outage for generator nr. 7: Medvode G2
Processing generator outage for generator nr. 8: Vrhovo G2
Processing generator outage for generator nr. 9: Bostanj G3
Processing generator outage for generator nr. 10: Blanca G3
Processing generator outage for generator nr. 11: HE Krško G1
Processing generator outage for generator nr. 12: Brezice G3
Processing generator outage for generator nr. 13: Dravograd G1
Processing generator outage for generator nr. 14: Vuzenica G2
Processing generator outage for generator nr. 15: TEB_PB7
Processing generator outage for generator nr. 16: TEŠ 6
Processing generator outage for generator nr